In [4]:
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_metric
import numpy as np

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#dataset = load_dataset("GSM8K", 'main')
#dataset

In [6]:
dataset = load_dataset("GSM8K", 'main', split={
    'train': 'train[:6000]',
    'test': 'test[:1319]'
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [7]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

#Moving the model to the gpu
model = model.to(device)

#setting up the pad token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
def preprocess_function(examples):
    inputs = ["solve: " + question for question in examples["question"]]
    targets = [answer for answer in examples["answer"]]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the entire dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./t5_small_gsm8k",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    per_device_eval_batch_size=1,  # Adjust based on GPU memory
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)



Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [9]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.267900,0.326793
2,0.368000,0.304405
3,0.328500,0.299140


TrainOutput(global_step=18000, training_loss=0.36647037084235085, metrics={'train_runtime': 1033.2588, 'train_samples_per_second': 17.421, 'train_steps_per_second': 17.421, 'total_flos': 2436152426496000.0, 'train_loss': 0.36647037084235085, 'epoch': 3.0})

In [12]:
# Example question
question = "What is the sum of 2 and 3?"

# Preprocess the question
input_text = "solve: " + question
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Generate the answer
output = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("Question:", question)
print("Generated Answer:", generated_answer)

Question: What is the sum of 2 and 3?
Generated Answer: The sum of 2 and 3 is 2 + 3 = 2+3=4>>4. The sum of 2 and 3 is 2 + 3 = 2+3=4>>4. #### 4


In [11]:
#END